In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.decomposition import PCA 
import featuretools as ft
# Regression Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Classification metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import cohen_kappa_score
# Ignore ConvergenceWarning messages
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
# smaller datasets for automated features
train     = pd.read_csv('application_train.csv')
bureau    = pd.read_csv('bureau.csv')
bureaubal = pd.read_csv('bureau_balance.csv')
prev      = pd.read_csv('previous_application.csv')
ccb       = pd.read_csv('credit_card_balance.csv')
insta     = pd.read_csv('installments_payments.csv')
pc        = pd.read_csv('POS_CASH_balance.csv')

In [9]:
train = train.sample(frac=1)
train = train[:30000]
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
6517,107601,0,Cash loans,F,N,Y,2,135000.0,484789.5,26428.5,418500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.022625,-10035,-3261,-2672.0,-2693,NaN,1,1,0,1,0,0,NaN,4.0,2,2,WEDNESDAY,17,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-712.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0
65672,176159,0,Cash loans,M,Y,N,0,225000.0,364896.0,28368.0,315000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.011657,-19784,-6767,-10471.0,-3309,15.0,1,1,1,1,0,0,Managers,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,NaN,0.0225,NaN,0.0132,NaN,block of flats,0.0272,"Stone, brick",No,0.0,0.0,0.0,0.0,-721.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
154139,278647,0,Cash loans,F,N,Y,0,144000.0,781920.0,25969.5,675000.0,Unaccompanied,Working,Secondary / secondary special,Separated,House / apartment,0.009657,-16429,-3685,-4274.0,-4629,NaN,1,1,0,1,0,0,Sales staff,1.0,2,2,SATURDAY,15,0,0,0,0,0,0,...,NaN,0.0972,NaN,0.0000,NaN,block of flats,0.0808,Panel,No,2.0,0.0,2.0,0.0,-201.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,5.0,2.0,2.0
107750,225007,0,Cash loans,M,Y,Y,0,292500.0,450000.0,16677.0,450000.0,Children,Commercial associate,Secondary / secondary special,Married,House / apartment,0.014464,-18207,-1134,-10245.0,-1698,17.0,1,1,0,1,0,0,Managers,2.0,2,2,TUESDAY,8,0,0,0,0,0,0,...,0.1513,0.0889,0.0078,0.0082,reg oper account,block of flats,0.0950,"Stone, brick",No,6.0,0.0,6.0,0.0,-1439.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,4.0,3.0,2.0
244070,382519,1,Cash loans,F,N,Y,1,247500.0,886176.0,34618.5,765000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.007020,-13052,-281,-2591.0,-4271,NaN,1,1,0,1,0,0,Sales staff,3.0,2,2,THURSDAY,14,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2243.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183725,312949,0,Revolving loans,M,N,N,0,112500.0,180000.0,9000.0,180000.0,Unaccompanied,Working,Higher education,Single / not married,House / apartment,0.009657,-12764,-2618,-6904.0,-4094,NaN,1,1,0,1,0,0,Laborers,1.0,2,2

In [10]:
ids = train['SK_ID_CURR'].values
bureau = bureau.loc[bureau['SK_ID_CURR'].isin(ids)]
idsb = bureau['SK_ID_BUREAU'].values
bureaubal = bureaubal.loc[bureaubal['SK_ID_BUREAU'].isin(idsb)]
prev = prev.loc[prev['SK_ID_CURR'].isin(ids)]
ccb  = ccb.loc[ccb['SK_ID_CURR'].isin(ids)]
insta = insta.loc[insta['SK_ID_CURR'].isin(ids)]
pc   = pc.loc[pc['SK_ID_CURR'].isin(ids)]
print('bureau shape: ', bureau.shape)
print('bureaubal shape: ', bureaubal.shape)
print('prev shape: ', prev.shape)
print('ccb shape: ', ccb.shape)
print('insta shape: ', insta.shape)
print('pc shape: ', pc.shape)

bureau shape:  (86296, 17)
bureaubal shape:  (931629, 3)
prev shape:  (137538, 37)
ccb shape:  (86203, 23)
insta shape:  (1131937, 8)
pc shape:  (86897, 8)


In [11]:
es = ft.EntitySet(id = 'applications')

In [12]:
es = es.entity_from_dataframe(entity_id = 'train'    , dataframe = train    , index= 'SK_ID_CURR')
es = es.entity_from_dataframe(entity_id = 'bureau'   , dataframe = bureau   , index= 'SK_ID_BUREAU')
es = es.entity_from_dataframe(entity_id = 'bureaubal', dataframe = bureaubal, make_index = True, index = 'bb_id')
es = es.entity_from_dataframe(entity_id = 'prev'     , dataframe = prev     , index = 'SK_ID_PREV')
es = es.entity_from_dataframe(entity_id = 'ccb'      , dataframe = ccb      , make_index = True, index = 'cc_id')
es = es.entity_from_dataframe(entity_id = 'insta'    , dataframe = insta    , make_index = True, index = 'installment.id')
es = es.entity_from_dataframe(entity_id = 'pc'       , dataframe = pc       , make_index = True, index = 'pos_cash_id')

In [13]:
# Relationship between application training and bureau
r_applications_bureau = ft.Relationship(es['train']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])
es = es.add_relationship(r_applications_bureau)
# Relationship between bureau and bureau balance
r_bureau_bureaubal = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureaubal']['SK_ID_BUREAU'])
es = es.add_relationship(r_bureau_bureaubal)
# Relationship between application training and previous applications
r_app_prev = ft.Relationship(es['train']['SK_ID_CURR'], es['prev']['SK_ID_CURR'])
es = es.add_relationship(r_app_prev)
# Relationship between previous applications with credit card balance, pos cash, and installments
r_prev_cc = ft.Relationship(es['prev']['SK_ID_PREV'], es['ccb']['SK_ID_PREV'])
es = es.add_relationship(r_prev_cc)
r_prev_insta = ft.Relationship(es['prev']['SK_ID_PREV'], es['insta']['SK_ID_PREV'])
es = es.add_relationship(r_prev_insta)
r_prev_pc2 = ft.Relationship(es['prev']['SK_ID_PREV'], es['pc']['SK_ID_PREV'])
es = es.add_relationship(r_prev_pc2)
es

Entityset: applications
  Entities:
    train [Rows: 30000, Columns: 122]
    bureau [Rows: 86296, Columns: 17]
    bureaubal [Rows: 931629, Columns: 4]
    prev [Rows: 137538, Columns: 37]
    ccb [Rows: 86203, Columns: 24]
    insta [Rows: 1131937, Columns: 9]
    pc [Rows: 86897, Columns: 9]
  Relationships:
    bureau.SK_ID_CURR -> train.SK_ID_CURR
    bureaubal.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    prev.SK_ID_CURR -> train.SK_ID_CURR
    ccb.SK_ID_PREV -> prev.SK_ID_PREV
    insta.SK_ID_PREV -> prev.SK_ID_PREV
    pc.SK_ID_PREV -> prev.SK_ID_PREV

In [14]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head()

,name,type,description
0,avg_time_between,aggregation,Computes the average time between consecutive events.
1,mean,aggregation,Computes the average value of a numeric feature.
2,sum,aggregation,Sums elements of a numeric or boolean feature.
3,trend,aggregation,Calculates the slope of the linear trend of variable overtime.
4,count,aggregation,Counts the number of non null values.


In [15]:
primitives[primitives['type'] == 'transform']

,name,type,description
19,cum_sum,transform,Calculates the sum of previous values of an instance for each value in a time-dependent entity.
20,absolute,transform,Absolute value of base feature.
21,months,transform,Transform a Timedelta feature into the number of months.
22,haversine,transform,Calculate the approximate haversine distance in miles between two LatLong variable types.
23,or,transform,"For two boolean values, determine if one value is 'True'."
24,weeks,transform,Transform a Timedelta feature into the number of weeks.
25,latitude,transform,Returns the first value of the tuple base feature.
26,minutes,transform,Transform a Timedelta feature into the number of minutes.
27,not,transform,"For each value of the base feature, negates the boolean value."
28,seconds,transform,Transform a Timedelta feature into the number of seconds.


In [16]:
features, feature_names = ft.dfs(entityset = es, target_entity = 'train',
                                  max_depth = 2)

In [17]:
features

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,MEAN(prev.STD(insta.SK_ID_CURR)),MEAN(prev.STD(insta.NUM_INSTALMENT_VERSION)),MEAN(prev.STD(insta.NUM_INSTALMENT_NUMBER)),MEAN(prev.STD(insta.DAYS_INSTALMENT)),MEAN(prev.STD(insta.DAYS_ENTRY_PAYMENT)),MEAN(prev.STD(insta.AMT_INSTALMENT)),MEAN(prev.STD(insta.AMT_PAYMENT)),MEAN(prev.MAX(insta.SK_ID_CURR)),MEAN(prev.MAX(insta.NUM_INSTALMENT_VERSION)),MEAN(prev.MAX(insta.NUM_INSTALMENT_NUMBER)),MEAN(prev.MAX(insta.DAYS_INSTALMENT)),MEAN(prev.MAX(insta.DAYS_ENTRY_PAYMENT)),MEAN(prev.MAX(insta.AMT_INSTALMENT)),MEAN(prev.MAX(insta.AMT_PAYMENT)),MEAN(prev.SKEW(insta.SK_ID_CURR)),MEAN(prev.SKEW(insta.NUM_INSTALMENT_VERSION)),MEAN(prev.SKEW(insta.NUM_INSTALMENT_NUMBER)),MEAN(prev.SKEW(insta.DAYS_INSTALMENT)),MEAN(prev.SKEW(insta.DAYS_ENTRY_PAYMENT)),MEAN(prev.SKEW(insta.AMT_INSTALMENT)),MEAN(prev.SKEW(insta.AMT_PAYMENT)),MEAN(prev.MIN(insta.SK_ID_CURR)),MEAN(prev.MIN(insta.NUM_INSTALMENT_VERSION)),MEAN(prev.MIN(insta.NUM_INSTALMENT_NUMBER)),MEAN(prev.MIN(insta.DAYS_INSTALMENT)),MEAN(prev.MIN(insta.DAYS_ENTRY_PAYMENT)),MEAN(prev.MIN(insta.AMT_INSTALMENT)),MEAN(prev.MIN(insta.AMT_PAYMENT)),MEAN(prev.MEAN(insta.SK_ID_CURR)),MEAN(prev.MEAN(insta.NUM_INSTALMENT_VERSION)),MEAN(prev.MEAN(insta.NUM_INSTALMENT_NUMBER)),MEAN(prev.MEAN(insta.DAYS_INSTALMENT)),MEAN(prev.MEAN(insta.DAYS_ENTRY_PAYMENT)),MEAN(prev.MEAN(insta.AMT_INSTALMENT)),MEAN(prev.MEAN(insta.AMT_PAYMENT)),MEAN(prev.COUNT(insta)),NUM_UNIQUE(prev.MODE(ccb.NAME_CONTRACT_STATUS)),NUM_UNIQUE(prev.MODE(pc.NAME_CONTRACT_STATUS)),MODE(prev.MODE(ccb.NAME_CONTRACT_STATUS)),MODE(prev.MODE(pc.NAME_CONTRACT_STATUS))
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,...,0.0,0.125988,2.545542,76.366269,76.380137,58313.693509,58313.693509,100003.0,1.333333,8.333333,-1054.333333,-1063.333333,210713.44500,210713.44500,0.0,0.881917,0.000000,0.000000,-0.029891,-1.089280,-1.089280,100003.0,1.000000,1.0,-1274.333333,-1283.000000,56431.86000,56431.8600,100003.0,1.047619,4.666667,-1164.333333,-1171.781746,78558.479286,78558.479286,8.333333,0.0,1.0,NaN,Active
100040,0,Cash loans,F,N,Y,0,135000.0,1125000.0,32895.0,1125000.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.019689,-15997,-4585,-5735.0,-4067,NaN,1,1,0,1,0,0,Core staff,2.0,2,2,FRIDAY,14,0,0,0,0,0,0,Security Ministries,...,0.0,0.188982,5.754570,172.637109,174.818712,10269.298511,12873.650738,100040.0,2.000000,21.000000,-407.000000,-417.000000,73324.26000,73324.26000,0.0,5.291503,0.280021,0.280021,0.296112,5.291503,3.235447,100040.0,1.000000,1.0,-1007.000000,-1017.000000,18984.24000,2.5200,100040.0,1.035714,10.178571,-731.642857,-740.214286,20924.955000,16178.895000,28.000000,0.0,1.0,NaN,Active
100105,0,Cash loans,F,N,Y,0,193500.0,225000.0,23755.5,225000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.031329,-19810,-8862,-4493.0,-3043,NaN,1,1,1,1,1,0,Managers,2.0,2,2,TUESDAY,11,0,0,0,0,0,0,Government,...,0.0,0.032141,1.856688,55.700642,56.019158,503.725797,4095.275209,100105.0,1.166667,5.583333,-772.416667,-776.916667,25389.9

In [18]:
# cols = features.select_dtypes(include = object).columns
# features[cols] = features[cols].fillna('Missing')
# cols = features.select_dtypes(exclude = object).columns
# features[cols] = features[cols].fillna(0)
# features.isnull().sum()

In [19]:
features.to_csv('train_ft.csv')